In [29]:
import yfinance as yf
import pandas as pd
from stock_config import momentum_stock_list 

## Input the evaluation date below (1st friday of every month)

In [30]:
eval_date = "2020-09-12"

#### Function to check valid trading day. If open price is not returned then next date is checked until valid date is found

In [31]:
def get_valid_date(start_date, end_date):
    valid = False
    while not valid:
        valid_check_data = yf.download('RELIANCE.NS', start = start_date, end=end_date)
        try:
            open_price = valid_check_data.iloc[0]['Open']
            valid = True
        except:
            start_date = (pd.to_datetime(start_date) + pd.DateOffset(days=1)).strftime('%Y-%m-%d')
            end_date = (pd.to_datetime(end_date) + pd.DateOffset(days=1)).strftime('%Y-%m-%d')
            valid = False
    return start_date, end_date

#### Get valid start and end dates for this year

In [32]:
start_date=eval_date
end_date = (pd.to_datetime(start_date) + pd.DateOffset(days=1)).strftime('%Y-%m-%d')
start_date, end_date = get_valid_date(start_date, end_date)

[*********************100%***********************]  1 of 1 completed


In [33]:
start_date

'2020-09-12'

In [34]:
end_date

'2020-09-13'

### Fetch this year data

#### [Below data fetch will throw error if there is no trading day between start and end dates]

In [35]:
data = yf.download(momentum_stock_list, start = start_date, end=end_date)

[*********************100%***********************]  493 of 493 completed


In [36]:
this_year_close = pd.DataFrame(data.iloc[0]['Adj Close'])

In [37]:
this_year_close

,2020-09-11
3MINDIA.NS,18569.000000
AARTIIND.NS,1022.299988
AAVAS.NS,1322.449951
ABB.NS,898.400024
ABBOTINDIA.NS,16819.250000
...,...
WIPRO.NS,293.299988
WOCKPHARMA.NS,291.149994
ZEEL.NS,218.100006
ZENSARTECH.NS,173.399994


#### Calculate dates for last year

In [38]:
start_date=(pd.to_datetime(eval_date) - pd.DateOffset(years=1)).strftime('%Y-%m-%d')
end_date = (pd.to_datetime(start_date) + pd.DateOffset(days=1)).strftime('%Y-%m-%d')

#### Get valid start and end dates for last year

In [39]:
start_date, end_date = get_valid_date(start_date, end_date)

[*********************100%***********************]  1 of 1 completed


In [40]:
start_date

'2019-09-12'

In [41]:
end_date

'2019-09-13'

### Fetch last year data

#### [Below data fetch will throw error if there is no trading day between start and end dates]

In [42]:
data = yf.download(momentum_stock_list, start = start_date, end=end_date)

[*********************100%***********************]  493 of 493 completed

8 Failed downloads:
- CDSL.NS: No data found for this date range, symbol may be delisted
- RELIANCE.NS: No data found, symbol may be delisted
- TORNTPOWER.NS: No data found, symbol may be delisted
- AEGISCHEM.NS: No data found, symbol may be delisted
- MAHABANK.NS: No data found, symbol may be delisted
- ICICIGI.NS: No data found, symbol may be delisted
- JUBILANT.NS: No data found, symbol may be delisted
- BANKBARODA.NS: No data found for this date range, symbol may be delisted


In [43]:
last_year_close = pd.DataFrame(data.iloc[0]['Adj Close'])

In [44]:
last_year_close

,2019-09-12
3MINDIA.NS,20142.099609
AARTIIND.NS,789.889709
AAVAS.NS,1541.599976
ABB.NS,1312.551758
ABBOTINDIA.NS,9597.436523
...,...
WIPRO.NS,243.566681
WOCKPHARMA.NS,324.299988
ZEEL.NS,345.930939
ZENSARTECH.NS,214.066025


In [45]:
final_df = last_year_close.join(this_year_close)

In [46]:
final_df

,2019-09-12,2020-09-11
3MINDIA.NS,20142.099609,18569.000000
AARTIIND.NS,789.889709,1022.299988
AAVAS.NS,1541.599976,1322.449951
ABB.NS,1312.551758,898.400024
ABBOTINDIA.NS,9597.436523,16819.250000
...,...,...
WIPRO.NS,243.566681,293.299988
WOCKPHARMA.NS,324.299988,291.149994
ZEEL.NS,345.930939,218.100006
ZENSARTECH.NS,214.066025,173.399994


In [47]:
final_df.rename(columns =lambda t: t.strftime('%Y-%m-%d'), inplace=True)

## Calculate Momentum

In [48]:
last_year_date = final_df.columns[0]
this_year_date = final_df.columns[1]
final_df['Change'] = final_df[this_year_date] - final_df[last_year_date]

In [49]:
final_df['% Change'] = (final_df['Change'] / final_df[last_year_date])*100

## Final Output

In [50]:
final_df.sort_values(by=['% Change'], ascending=False, inplace=True)

In [51]:
final_df.rename_axis('MOMENTUM_ATH', inplace=True)
final_df.rename_axis('', axis='columns', inplace=True)

In [52]:
final_df.round(2).head(50)

,2019-09-12,2020-09-11,Change,% Change
MOMENTUM_ATH,,,,
ADANIGREEN.NS,48.90,608.65,559.75,1144.68
ALKYLAMINE.NS,781.90,3184.90,2403.00,307.33
GMMPFAUDLR.NS,1462.51,5599.45,4136.94,282.87
LAURUSLABS.NS,341.22,1257.75,916.53,268.60
DIXON.NS,2524.36,9240.00,6715.64,266.03
AFFLE.NS,857.75,2907.95,2050.20,239.02
GRANULES.NS,103.37,344.30,240.93,233.06
INDIAMART.NS,1611.50,4768.00,3156.50,195.87
DEEPAKNTR.NS,270.43,733.15,462.72,171.11


In [53]:
filename = f'data/output/momentum_yearly_output_{eval_date}.xlsx'

In [54]:
final_df.round(2).to_excel(filename)

In [55]:
f"Done. Output file path and name --> {filename}"

'Done. Output file path and name --> data/output/momentum_yearly_output_2020-09-12.xlsx'